<font>
<div dir=ltr align=center>
<img src='../Utils/Sharif_logo.png' width=150 height=150> <br>
<font color=0F5298 size=7>
Regression Analysis<br>
<font color=2565AE size=5>
Summer 2025<br>
<font color=3C99D size=5>
Project 2 - Data Preparation <br>
<font color=696880 size=4>

| Name              | ID        | Major              |
|-------------------|-----------|--------------------|
| 👨🏻 Ali M. Shabestari | 401106482 | Computer Engineering |
| 🧔🏻‍♂️ Arshia Dehghan   | 401100382 | Computer Science     |
| 🧔🏻 Abolfazl Moslemi   | 401100506 | Computer Science     |

In [1]:
import pandas as pd
import numpy as np
paths = ['Tehran_1399', 'Tehran_1400', 'Tehran_1401']

In [ ]:
import zipfile
import glob

zip_files = glob.glob("*.zip")

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zf:
        zf.extractall(".")

In [2]:
import pandas as pd

def head_columns(path):
    path = path + '/members_properties_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    heads = df[df['Relationship'] == 'Head'].copy()
    heads['Head_Age_'] = np.minimum(heads['Age'], 90)    
    heads = heads.rename(columns={
        'Age': 'Head_Age',
        'Sex': 'Head_Sex',
        'Education_Level': 'Head_Education_Level', 
        'Activity_Status': 'Head_Activity_Status',
        'Marital_Status': 'Head_Marital_Status'
    })
    # Binning Age
    bins = [0, 20, 30, 40, 50, 60, 70, float('inf')]
    labels = ['Below 20', '20-30', '30-40', '40-50', '50-60', '60-70', 'Above 70']
    heads['Head_Age'] = pd.cut(heads['Head_Age'], bins=bins, labels=labels, right=False)
    
    # Mapping Education
    education_mapping = {
    'Unofficial': 'Under_Diploma',
    'Primary': 'Under_Diploma',
    'Lower_Secondary': 'Under_Diploma',
    'Higher_Secondary': 'Under_Diploma',
    'Pre_University': 'Diploma',
    'Short_Cycle_Tertiary': 'Bachelor',
    'Bachelors': 'Bachelor',
    'Masters': 'Masters_PhD',
    'Doctoral': 'Masters_PhD'
}
    heads['Head_Education_Level'] = heads['Head_Education_Level'].map(education_mapping).fillna('Illiterate')
    heads = heads[heads['Head_Activity_Status'] != 'Student']
    return heads[['Year', 'ID', 'Head_Age', 'Head_Age_', 'Head_Sex', 'Head_Education_Level', 'Head_Activity_Status', 'Head_Marital_Status']]

In [3]:
def family_size(path):
    path = path + '/Number_of_Members_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    df = df.rename(columns={'Members': 'Household_Size'})
    df['Household_Size'] = df['Household_Size'].apply(
        lambda x: 'Above 7' if x > 7 else x
    )
    family_size = df[['Year', 'ID', 'Household_Size']]
    return family_size

In [4]:
def income_share(path):
    path = path + '/Income_Breakdown_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    income_groups = {
        'Cash_Private': 'Labor_Income',
        'Cash_Public': 'Labor_Income',
        'Cash_NonAgricultural': 'Labor_Income',
        'Cash_Agricultural': 'Labor_Income',
        'Cash_Home_Production': 'Labor_Income',
        'Cash_Cooperative': 'Labor_Income',

        'Cash_Retirement': 'Pension_Income',

        'Cash_Rent': 'Capital_Income',
        'Cash_Interest': 'Capital_Income',

        'Cash_Subsidy': 'Aid_Income',
        'Cash_Aid': 'Aid_Income',
        'Cash_Transfer': 'Aid_Income',
        'NonCash_Donation': 'Aid_Income',
        'NonCash_Public': 'Aid_Income',

        'NonCash_ImputedRent_Ownership': 'Imputed_Rent',
        'NonCash_ImputedRent_Mortgage': 'Imputed_Rent',
        
        'NonCash_Private': 'Other_NonCash_Income',
        'NonCash_NonAgricultural': 'Other_NonCash_Income',
        'NonCash_Agricultural': 'Other_NonCash_Income',
        'NonCash_HomeProduction': 'Other_NonCash_Income',
        'NonCash_Cooperative': 'Other_NonCash_Income'
    }

    df['Income_Group'] = df['Income_Type'].map(income_groups)

    agg_df = df.groupby(['Year', 'ID', 'Income_Group'])['Income'].sum().unstack(fill_value=0)

    agg_df['Total_Income'] = agg_df.sum(axis=1)
    for col in ['Labor_Income', 'Pension_Income', 'Capital_Income', 'Aid_Income', 'Imputed_Rent', 'Other_NonCash_Income']:
        agg_df[f'{col}_Share'] = round(agg_df[col] / agg_df['Total_Income'], 2)

    agg_df = agg_df.reset_index()
    return agg_df[['Year', 'ID', 'Aid_Income_Share', 'Labor_Income_Share', 'Pension_Income_Share', 'Capital_Income_Share', 'Imputed_Rent_Share', 'Other_NonCash_Income_Share']]

In [5]:
def weight(path):
    path = path + '/household_information_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    df = df.rename(columns={'Weight': 'Household_Weight'})
    return df[['Year', 'ID', 'Household_Weight']]

In [6]:
def net_expenditures(path):
    path = path + '/Total_Expenditure_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    return df[['Year', 'ID', 'Net_Expenditure']]

In [7]:
def oecd_modified(path):
    path = path + '/Equivalence_Scale_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    return df[['Year', 'ID', 'OECD_Modified']]

In [8]:
def dependency_ratios(path):
    path = path + '/members_properties_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)

    # Create flag columns
    df['is_employed'] = df['Activity_Status'] == 'Employed'
    df['is_not_employed'] = ~df['is_employed']
    df['is_working_age'] = df['Age'].between(15, 64)
    df['is_dependent_age'] = ~df['is_working_age']

    # Group by ID
    grouped = df.groupby('ID').agg(
        employed_count=('is_employed', 'sum'),
        not_employed_count=('is_not_employed', 'sum'),
        working_age_count=('is_working_age', 'sum'),
        dependent_age_count=('is_dependent_age', 'sum')
    )

    # Compute age-based dependency ratio
    grouped['working_age_count'] = grouped['working_age_count'].apply(lambda x: 1 if x == 0 else x)
    grouped['employed_count'] = grouped['employed_count'].apply(lambda x: 1 if x == 0 else x)

    grouped['Dependency_Ratio_Age'] = grouped['dependent_age_count'] / grouped['working_age_count']
    grouped['Dependency_Ratio_Age'] = grouped['Dependency_Ratio_Age'].round(2)
    grouped['Dependency_Ratio_Employment'] = (grouped['not_employed_count'] / grouped['employed_count'])
    grouped['Dependency_Ratio_Employment'] = grouped['Dependency_Ratio_Employment'].round(2)

    # Reset index to include ID in output
    grouped = grouped.reset_index()

    return grouped[['ID', 'Dependency_Ratio_Age', 'Dependency_Ratio_Employment']]

In [9]:
def tenure(path):
    path = path + '/house_specifications_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    df = df.rename(columns={'Tenure': 'Tenure_Type'})
    tenure_mapping = {
    'Owned_Estate_Land': 'Owner',
    'Mortgage': 'Owner',
    'Rent': 'Tenant',
    'Free': 'Other',
    'Service': 'Other'
}
    df['Tenure_Type'] = df['Tenure_Type'].map(tenure_mapping)
    return df[['Year', 'ID', 'Tenure_Type']]

In [10]:
def urban(path): 
    path = path + '/Administrative_Divisions_' + path.split('_')[-1] + '.csv'
    df = pd.read_csv(path)
    df = df.rename(columns={'Urban_Rural_Name': 'Urban_Rural'})
    return df[['Year', 'ID', 'Urban_Rural']]

In [11]:
for path in paths:
    heads = head_columns(path)
    family = family_size(path)
    income = income_share(path)
    weight_df = weight(path)
    expenditures = net_expenditures(path)
    oecd = oecd_modified(path)
    dep_ratio = dependency_ratios(path)
    tenure_type = tenure(path)
    urban_rural = urban(path)

    df = pd.merge(heads, family, on=['Year', 'ID'])
    df = pd.merge(df, income, on=['Year', 'ID'])
    df = pd.merge(df, weight_df, on=['Year', 'ID'])
    df = pd.merge(df, expenditures, on=['Year', 'ID'])
    df = pd.merge(df, oecd, on=['Year', 'ID'])
    df = pd.merge(df, dep_ratio, on='ID')
    df = pd.merge(df, tenure_type, on=['Year', 'ID'])
    df = pd.merge(df, urban_rural, on=['Year', 'ID'])

    df.to_csv(f'{path}_prepared.csv', index=False)

In [12]:
import pandas as pd

# List of CSV file paths to stack
csv_files = [
    'Tehran_1399_prepared.csv',
    'Tehran_1400_prepared.csv',
    'Tehran_1401_prepared.csv'
]

# Read and concatenate all CSVs
df_all = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Show the shape of the combined dataframe
df_all.to_csv('Tehran_all_prepared.csv', index=False)